<a href="https://colab.research.google.com/github/avikumart/LLM-GenAI-Transformers-Notebooks/blob/main/DeepLearningFiles/mental_health_convo_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/nlp-mental-health-conversations")

print("Path to dataset files:", path)

100%|██████████| 1.48M/1.48M [00:00<00:00, 112MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/thedevastator/nlp-mental-health-conversations/versions/2


In [ ]:
import pandas as pd
import numpy as np
import os

df = pd.read_csv(os.path.join(path, "train.csv"))
df.head()

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [ ]:
# what are the unique values in the Contexrt column
df['Context'].unique()

array(["I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?",
       'I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?',
       'I have been feeling more and more down for over a month. I have started having trouble sleeping due to panic attacks, but they are almost never triggered by something that I know of.',
       'I’m facing severe depression and anxiety and I just feel like I’m going

In [ ]:
df["Context"].nunique()

995

In [ ]:
df["Response"].nunique()

2479

In [ ]:
# finetuning the OS model
!pip install transformers accelerate peft bitsandbytes datasets trl -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.9 MB/s eta 0:00:00


In [ ]:
# from huggingface_hub import login
# login()

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "google/gemma-2-2b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset
text = "You are a helpful mental health assistant answering following input to the output"
df["instruction"] = text
df["input"] = df["Context"]
df["output"] = df["Response"]
ndf = df[["instruction","input", "output"]]
ndf.head()

,instruction,input,output
0,You are a helpful mental health assistant answ...,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,You are a helpful mental health assistant answ...,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,You are a helpful mental health assistant answ...,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,You are a helpful mental health assistant answ...,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,You are a helpful mental health assistant answ...,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [ ]:
# Convert the dataset into a list of dictionaries
dataset = Dataset.from_pandas(ndf)
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 3512
})

In [ ]:
# train and test split for the dataset
dataset = dataset.train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 2809
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 703
    })
})

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig

# Quantization config for memory efficiency
#bnb_config = BitsAndBytesConfig(
#    load_in_4bit=True,
#    bnb_4bit_quant_type="nf4",
#    bnb_4bit_compute_dtype="float16",
#)

# Load model with quantization
#model = AutoModelForCausalLM.from_pretrained(
#    model_name,
#    device_map="auto",
#)

# Prepare model for training
model = prepare_model_for_kbit_training(model)

# LoRA configuration
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,  # Alpha parameter
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Target attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA
model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 6,389,760 || all params: 2,620,731,648 || trainable%: 0.2438


In [ ]:
# train the model on custom dataset
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "./gemma-finetuned-mhfaq",
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    learning_rate = 2e-4,
    logging_steps = 10,
    max_steps = 100,
    num_train_epochs=3,
    weight_decay = 0.01,
    fp16 = True,
    bf16 = False,
    max_grad_norm = 0.3,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type = "constant",
    report_to = "tensorboard",
    save_strategy="epoch"
)

# define the dataset formatting function
def formatting(example):
  text = f"Instrcution: {example['instruction']}\n"
  if example.get('input'):
    text += f"Input: {example['input']}\n"
  text += f"Output: {example['output']}"
  return text

# define the trainer
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset["train"],
    eval_dataset = dataset["test"],
    formatting_func = formatting,
    args = training_args,
    peft_config = lora_config,
)

#train the model
trainer.train()

Applying formatting function to train dataset:   0%|          | 0/2809 [00:00<?, ? examples/s]

Converting train dataset to ChatML:   0%|          | 0/2809 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/2809 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2809 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2809 [00:00<?, ? examples/s]